# Crypto currency

- Retrieve historical data of a crypto currency (Bitcoin)
    - https://min-api.cryptocompare.com/data/histominute?fsym=BTC&tsym=USD&limit=10
- Find a way to update this historical data
- Plot crypto currency in an interactive manner

## 1. Historical data retrievement

The API used to retrieve historical data is **CryptoCompare** https://min-api.cryptocompare.com/. We chose this API because it provides many benefits for free :
1. Retrieve historical data at minute interval up to 7 days. For older times, interval becomes hourly or daily. 
2. Many crypto currencies available TODO give a list
3. Easy to use, no need to authenticate
4. High rate limits : https://min-api.cryptocompare.com/stats/rate/limit

In [29]:
import urllib.request
import pandas as pd
import json
import numpy as np
import datetime
import time
import os

## API config
BASE_URL         = "https://min-api.cryptocompare.com/data/histominute" # API base url
CURRENCY         = "USD" # Available currency in the API
APP_NAME         = "HES_SO_master_crypto_analysis" # API advises us to give an app name in requests

### CHANGE THIS PARAMETER IF YOU WANT DIFFERENT CRYPTO 
CRYPTO_SYMBOL    = 'ZIL' #'BTC' # Crypto currency retrieved from API
MAX_LIMIT        = 2000 # maximum data retrieved per API request

## personal config
FILE_PATH        = f"data/crypto/{CRYPTO_SYMBOL}" # Relative path to historical data
SEP_CHAR         = '~' # character seperating dates from and to in filename
ENVS             = ['CRYPTO', 'MOST_RECENT_TS', 'MOST_RECENT_FILE', 'MOST_RECENT_FILE_LINE_COUNT'] # Stored in var.csv
MAX_ROW_PER_FILE = 10000 # Each file storing data has a maximum amount of rows

In [30]:
def get_data_minutely(toTs, limit, crypto_symbol):
    '''
    Get data minutely
    
    --- params
    @ toTs = the data to return until this time. set to -1 if you want to retrieve most recent data
    @ limit = number of minutes to retrieve from 1 to 2000
    @ crypto_symbol = the symbol of cryptocurrency (e.g. 'BTC' for bitcoin)
    '''
    #if toTs < 0:
    #    contents = urllib.request.urlopen(
    #        f"{BASE_URL}?fsym={crypto_symbol}&tsym={CURRENCY}&limit={limit}&extraParams={APP_NAME}"
    #    ).read()
    #else:
    contents = urllib.request.urlopen(
        f"{BASE_URL}?fsym={crypto_symbol}&tsym={CURRENCY}&limit={limit}&toTs={toTs}&extraParams={APP_NAME}"
    ).read()
    json_string = contents.decode("utf-8")
    obj = json.loads(json_string)
    df = pd.DataFrame.from_dict(obj['Data'])
    if not df.empty:
        return df.drop(['volumefrom', 'volumeto'], axis=1)
    return df

def get_var(key, crypto):
    df_var = pd.read_csv("var.csv", sep=',',
                         dtype={'MOST_RECENT_TS': np.int32, 'MOST_RECENT_FILE_LINE_COUNT': np.int32})
    return df_var[key].loc[df_var['CRYPTO'] == crypto].values[0]

def update_var(key, value, crypto):
    df_var = pd.read_csv("var.csv", sep=',',
                         dtype={'MOST_RECENT_TS': np.int32, 'MOST_RECENT_FILE_LINE_COUNT': np.int32})
    df_var[key].loc[df_var['CRYPTO'] == crypto] = str(value)
    df_var.to_csv("var.csv", index=False)
    
def add_new_crypto(crypto):
    df_var = pd.read_csv("var.csv", sep=',',
                         dtype={'MOST_RECENT_TS': np.int32, 'MOST_RECENT_FILE_LINE_COUNT': np.int32})
    if df_var[ENVS[0]].loc[df_var['CRYPTO'] == crypto].empty:
        new_line = pd.DataFrame([[crypto,-1,"",0]], columns=ENVS)
        df_var = df_var.append(new_line)
        df_var.to_csv("var.csv", index=False)

In [31]:
add_new_crypto(CRYPTO_SYMBOL)

### 1.1 Data understanding

The data returned contains the following information :

- time : unix timestamp when data was recorded
- open : The price at opening time of the interval
- close : The price at close time of the interval
- low : The lowest price observed during the interval
- high : The higher price reached during the interval
- volumefrom : TODO
- volumeto : TODO

In [32]:
df_histo = get_data_minutely(-1, 10, CRYPTO_SYMBOL)
print(df_histo['time'].iloc[-1])
print(datetime.datetime.fromtimestamp(int(df_histo['time'].iloc[0])).strftime('%Y-%m-%d %H:%M:%S'))
print(datetime.datetime.fromtimestamp(int(df_histo['time'].iloc[-1])).strftime('%Y-%m-%d %H:%M:%S'))
print(df_histo.shape)
df_histo.head(3)

1527848460
2018-06-01 12:11:00
2018-06-01 12:21:00
(11, 5)


,close,high,low,open,time
0,0.1117,0.1118,0.1116,0.1116,1527847860
1,0.1117,0.1117,0.1117,0.1117,1527847920
2,0.1117,0.1117,0.1117,0.1117,1527847980


### 1.2 Collect and save data

Data is collected from the API and then saved in csv files. Each csv file contains a maximum amount of rows. We split the data into multiple files if this max amount is reached.

The API have limitations depending the time interval between two currencies. The more precise route for historical data let us retrieve currencies every minute but is limited to 7 days in the past. To retrieve more data back in the past, we must retrieve data hourly or daily.

In [33]:
# Nummber of objects wished to be retrieved from API. 
# If using minutely, then it corresponds to the number of minutes
total_wished = 60 * 24 * 7 # API LIMIT is 7 days minutely which is 60 * 24 * 7 minutes

# Last unix timestamp to return data for
toTs = -1

# Init dataframe of historical data
df_historical = pd.DataFrame()

# Retrieve existing most recent file if any
# If any file is present and it is not full, we'll use it to store future data
last_ts = get_var(ENVS[1], CRYPTO_SYMBOL)
last_file_line_cnt = 0
last_file = None
if last_ts > 0:
    last_file = get_var(ENVS[2], CRYPTO_SYMBOL)
    last_file_line_cnt = get_var(ENVS[3], CRYPTO_SYMBOL)
    now = int(time.time())
    total_wished = int((now - last_ts)/60) # Get data we didn't retrieve from now until last time

# Retrieve data for the 7 past days until last_ts
while(total_wished > 0):
    limit = min(total_wished, MAX_LIMIT)
    df = get_data_minutely(toTs, limit, CRYPTO_SYMBOL)
    #if df.empty:
    #    print(df.shape)
    #    break
    df_historical = df_historical.append(df)
    min_time = df['time'].iloc[0]
    max_time = df['time'].iloc[-1]
    print('range=', datetime.datetime.fromtimestamp(min_time).strftime('%Y-%m-%d %H:%M:%S'),
         datetime.datetime.fromtimestamp(max_time).strftime('%Y-%m-%d %H:%M:%S'))
    toTs = min_time
    total_wished = total_wished - df.shape[0]

# Sort data in ascending time
if not df_historical.empty:

    df_historical = df_historical.sort_values(by=['time'])

    ## Save historical data to csv
    available_lines = MAX_ROW_PER_FILE - last_file_line_cnt
    df_to_fulfill = df_historical.head(available_lines)
    df_remaining  = df_historical.tail(df_historical.shape[0] - df_to_fulfill.shape[0])

    print('df_to_fulfill', df_to_fulfill.shape)
    print('df_remaining', df_remaining.shape)

    # Fulfill last file if any
    from_str = datetime.datetime.fromtimestamp(df_to_fulfill['time'].iloc[0]).strftime('%Y-%m-%d %H-%M-%S')
    max_time = df_to_fulfill['time'].iloc[-1]
    to_str = datetime.datetime.fromtimestamp(max_time).strftime('%Y-%m-%d %H-%M-%S')
    if last_file is None:
        last_file = f"{FILE_PATH}/{from_str}{SEP_CHAR}{to_str}.csv"
        df_to_fulfill.to_csv(last_file, mode='w', header=True, index=False)
    else:
        df_to_fulfill.to_csv(last_file, mode='a', header=False, index=False)
        filename = f"{last_file.split(SEP_CHAR)[0]}{SEP_CHAR}{to_str}.csv"
        os.rename(last_file, filename)
    last_file_line_cnt = last_file_line_cnt + df_to_fulfill.shape[0]
        
    # Split remaining dataframe into chunk of MAX_ROW_PER_FILE rows max
    list_df_remaining = [df_remaining[i:i+MAX_ROW_PER_FILE] for i in range(0, df_remaining.shape[0], MAX_ROW_PER_FILE)]
    for df_remain in list_df_remaining:
        from_str = datetime.datetime.fromtimestamp(df_remain['time'].iloc[0]).strftime('%Y-%m-%d %H-%M-%S')
        max_time = df_remain['time'].iloc[-1]
        to_str = datetime.datetime.fromtimestamp(max_time).strftime('%Y-%m-%d %H-%M-%S')
        filename = f"{FILE_PATH}/{from_str}{SEP_CHAR}{to_str}.csv"
        df_remain.to_csv(filename, index = False)
        last_file_line_cnt = df_remain.shape[0]

    # Update env variables
    update_var(ENVS[1], max_time, CRYPTO_SYMBOL)
    update_var(ENVS[2], filename, CRYPTO_SYMBOL)
    update_var(ENVS[3], last_file_line_cnt, CRYPTO_SYMBOL)

print('done')

range= 2018-05-31 03:01:00 2018-06-01 12:21:00
range= 2018-05-29 17:41:00 2018-05-31 03:01:00
range= 2018-05-29 13:50:00 2018-05-29 17:41:00
df_to_fulfill (4171, 5)
df_remaining (63, 5)
done


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## 2. Plot crypto currency time series data

Read and plot retrieved data

In [34]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True) # plotly offline mode

import glob

MAX_ROW = 500 # Plot the MAX_ROW most recent points

### 2.1 Read data from csv files

For this example we'll collect max 1000 points (the most recent ones)

In [35]:
def get_most_recent_data_from_csv(folder, n_rows):
    '''
    @ folder : relative path where the data is contained
    @ n_rows : the number of rows to retrieve up to the most recent ones
    
    Return a dataframe containing n_rows of the most recent data retrieved
    '''
    files =  glob.glob(f"{folder}/*.csv")
    files = sorted(files)
    df = pd.DataFrame()
    for file in reversed(files):
        print(file)
        df = df.append(pd.read_csv(file))
        if df.shape[0] > n_rows:
            break
    return df.sort_values(by=['time']).tail(n_rows)

In [36]:
# Read MAX_POINTS from the last file(s)
df_hist_minutes = get_most_recent_data_from_csv(FILE_PATH, MAX_ROW)
print('df shape', df_hist_minutes.shape)
from_str = datetime.datetime.fromtimestamp(df_hist_minutes['time'].iloc[0]).strftime('%Y-%m-%d %H-%M-%S')
to_str = datetime.datetime.fromtimestamp(df_hist_minutes['time'].iloc[-1]).strftime('%Y-%m-%d %H-%M-%S')
print('from', from_str, 'to', to_str)
df = df_hist_minutes
df['time'] = df_hist_minutes['time'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
df.head(5)

data/crypto/ZIL/2018-06-01 11-19-00~2018-06-01 12-21-00.csv
data/crypto/ZIL/2018-05-25 12-43-00~2018-06-01 11-18-00.csv
df shape (500, 5)
from 2018-06-01 04-02-00 to 2018-06-01 12-21-00


,close,high,low,open,time
9563,0.1130,0.1131,0.1130,0.1131,2018-06-01 04:02:00
9564,0.1132,0.1132,0.1132,0.1132,2018-06-01 04:03:00
9565,0.1132,0.1132,0.1132,0.1132,2018-06-01 04:04:00
9566,0.1132,0.1132,0.1132,0.1132,2018-06-01 04:05:00
9567,0.1132,0.1132,0.1132,0.1132,2018-06-01 04:06:00


### 2.2 Plot candlestick like graph

Plot only the most recent points to see if evereything did well.

In [37]:
trace = go.Candlestick(x=df.time,
                       open=df.open,
                       high=df.high,
                       low=df.low,
                       close=df.close)
data = [trace]
layout = {
    'title': f"{CRYPTO_SYMBOL} currency",
    'yaxis': {'title': 'Price USD'},
}
fig = dict(data=data, layout=layout)
iplot(fig)